In this notebook, we want to:

- investigate the spatial structure of the residual stream
- see which tokens the different directions in the residual stream map to

# setup


In [ ]:
# Generic
import os
from pathlib import Path
from copy import deepcopy
import typing
import itertools
from collections import Counter

# Numerical Computing
import numpy as np
import torch
import pandas as pd
# import torch.nn.functional as F
from fancy_einsum import einsum
import einops
from jaxtyping import Float, Int, Bool

# scipy
from scipy.spatial.distance import pdist, squareform
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
# from scipy.spatial.distance import cosine

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets


# muutils
from muutils.misc import shorten_numerical_to_str
from muutils.nbutils.configure_notebook import configure_notebook
# TransformerLens imports
from transformer_lens import HookedTransformer, ActivationCache

# Our Code
from maze_dataset import MazeDataset, MazeDatasetConfig, SolvedMaze, LatticeMaze, SPECIAL_TOKENS
from maze_dataset.tokenization import MazeTokenizer, TokenizationMode
from maze_dataset.plotting.print_tokens import color_maze_tokens_AOTP
from maze_dataset.tokenization.token_utils import strings_to_coords, coords_to_strings
from maze_dataset.constants import _SPECIAL_TOKENS_ABBREVIATIONS

from maze_transformer.training.config import ConfigHolder, ZanjHookedTransformer, BaseGPTConfig
from muutils.tensor_utils import string_dict_shapes
from maze_transformer.mechinterp.plot_weights import plot_embeddings
from maze_transformer.mechinterp.residual_stream_structure import (
    TokenPlottingInfo, process_tokens_for_pca, EmbeddingsPCAResult, compute_pca, plot_pca_colored, 
    compute_distances_and_correlation, compute_grid_distances,
    plot_distances_matrix, plot_distance_grid, plot_distance_correlation,
)


In [ ]:
# Setup (we won't be training any models)
DEVICE: torch.device = configure_notebook(seed=42, dark_mode=False)
print(f"{DEVICE = }")
torch.set_grad_enabled(False)


In [ ]:
# path to load the model from
MODEL_PATH: str = "../examples/hallway-medium_2023-06-16-03-40-47.iter_26554.zanj"
# MODEL_PATH: str = "../examples/wandb.jerpkipj.pt.zanj"
# load the model and tokenizer
MODEL: ZanjHookedTransformer|HookedTransformer = ZanjHookedTransformer.read(MODEL_PATH)
num_params: int = MODEL.num_params()
print(f"loaded model with {shorten_numerical_to_str(num_params)} params ({num_params = }) from\n{MODEL_PATH}")
TOKENIZER: MazeTokenizer = MODEL.zanj_model_config.maze_tokenizer
print(f"using tokenizer with {TOKENIZER = }")
print(f"{TOKENIZER.vocab_size = }")

In [ ]:
# get tokenizer and embedding info
print(f"{TOKENIZER.token_arr = }")
d_model: int = MODEL.config.d_model
print(f"{MODEL.W_E.shape = }")
assert MODEL.W_E.shape == (TOKENIZER.vocab_size, d_model)
VOCAB_TOKENS: Int[torch.Tensor, "vocab_size"] = torch.arange(TOKENIZER.vocab_size, device=DEVICE)
assert VOCAB_TOKENS.tolist() == TOKENIZER.encode(TOKENIZER.token_arr)

# maps token to (index, coordinate)
COORDINATE_TOKENS_INFO: dict[str, (int, tuple[int,int])] = {
    tok: coord
    for idx, (tok, coord) in enumerate(zip(TOKENIZER.token_arr, TOKENIZER.strings_to_coords(TOKENIZER.token_arr, when_noncoord="include")))
    if not isinstance(coord, str)
}
print(f"{COORDINATE_TOKENS_INFO = }")

# information for how to plot the tokens
VOCAB_PLOT_INFO: list[TokenPlottingInfo] = process_tokens_for_pca(TOKENIZER)

In [ ]:
# MODEL.process_weights_(fold_ln=False, center_writing_weights=False, center_unembed=False)
# MODEL.process_weights_(fold_ln=False, center_writing_weights=False, center_unembed=True)
# MODEL.process_weights_(fold_ln=False, center_writing_weights=True, center_unembed=False)
# MODEL.process_weights_(fold_ln=False, center_writing_weights=True, center_unembed=True)
# MODEL.process_weights_(fold_ln=True, center_writing_weights=False, center_unembed=False)
# MODEL.process_weights_(fold_ln=True, center_writing_weights=False, center_unembed=True)
# MODEL.process_weights_(fold_ln=True, center_writing_weights=True, center_unembed=False)
# MODEL.process_weights_(fold_ln=True, center_writing_weights=True, center_unembed=True)

# embedding distances


In [ ]:
# possible distance functions:
"""
'braycurtis', 'canberra', 'chebyshev', 'cityblock',
'correlation', 'cosine', 'dice', 'euclidean', 'hamming',
'jaccard', 'jensenshannon', 'kulczynski1',
'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto',
'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath',
'sqeuclidean', 'yule'
"""

MODEL_EMBEDDING_MATRIX: Float[np.ndarray, "d_vocab d_model"] = MODEL.W_E.detach().cpu().numpy()

distances_result: dict = compute_distances_and_correlation(
    embedding_matrix=MODEL_EMBEDDING_MATRIX,
    tokenizer=TOKENIZER,
    embedding_metric="cosine",
    coordinate_metric="cityblock",
)

In [ ]:
# print info about 'correlation', 'corr_pval', 'tokenizer', 'embedding_metric', 'coordinate_metric'
print(
    f"""Correlation between embedding (via {distances_result['embedding_metric']}) and coordinate (via {distances_result['coordinate_metric']}) distances:
{distances_result['correlation']:.3f} (p={distances_result['corr_pval']:.3f})"""
)

In [ ]:
plot_distances_matrix(**distances_result, show=False)
plt.show()

In [ ]:
distance_grid: Float[np.ndarray, "n n n n"] = compute_grid_distances(
    embedding_distances_matrix=distances_result["embedding_distances_matrix"],
    tokenizer=TOKENIZER,
)

In [ ]:
plot_distance_correlation(
    distance_grid,
    embedding_metric=distances_result["embedding_metric"],
    coordinate_metric=distances_result["coordinate_metric"],
)

In [11]:
plot_distance_grid(distance_grid, embedding_metric=distances_result["embedding_metric"], show=False)
plt.show()


# PCA of token embeddings

In [ ]:
raise Exception("dont run the rest for now")

In [ ]:
# compute the PCA
PCA_RESULTS: dict[str, EmbeddingsPCAResult] = compute_pca(
    model=MODEL,
    token_plotting_info=VOCAB_PLOT_INFO,
)

In [ ]:
ipywidgets.interact(
    plot_pca_colored, 
    pca_results_options=ipywidgets.fixed(PCA_RESULTS),
    pca_results_key=ipywidgets.Dropdown(
        options=list(PCA_RESULTS.keys()),
        value='all',
        description='PCA Results:',
    ),
    vocab_colors=ipywidgets.fixed(VOCAB_PLOT_INFO), 
    dim1=ipywidgets.IntText(
        value=1,
        description='Dim 1:',
    ),
    dim2=ipywidgets.IntText(
        value=2,
        description='Dim 1:',
    ),
    lattice_connections=ipywidgets.Dropdown(
        options=[True, False],
        value=True,
        description='Show Lattice:',
    ),
    # float or None (entry box of float or text)
    symlog_scale=ipywidgets.FloatText(
        value=-1,
        description='Symlog Scale:',
    ),
    axes_and_centered=ipywidgets.Dropdown(
        options=[True, False],
        value=True,
        description='Center and show axes:',
    ),
);